In [27]:
import pandas as pd
import csv

In [12]:
KEY = "JP-T0oiXNqqLi-sMu1_hD-9v1c95-4fMBI-4fI1clL5_gJeAp_oRR1GHatW3tyW3-PmCI7px8cDrDdXF8Yxr-83QkltoeYwmk-viJp7oCYQalGm-bY-V79pDbi_NWnYx"

In [13]:
"""
Yelp Fusion API code sample.
This program demonstrates the capability of the Yelp Fusion API
by using the Search API to query for businesses by a search term and location,
and the Business API to query additional information about the top result
from the search query.
Please refer to http://www.yelp.com/developers/v3/documentation for the API
documentation.
This program requires the Python requests library, which you can install via:
`pip install -r requirements.txt`.
Sample usage of the program:
`python sample.py --term="bars" --location="San Francisco, CA"`
"""
from __future__ import print_function

import argparse
import json
import pprint
import requests
import sys
import urllib


# This client code can run on Python 2.x or 3.x.  Your imports can be
# simpler if you only need one of those.
try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
    from urllib import urlencode


# Yelp Fusion no longer uses OAuth as of December 7, 2017.
# You no longer need to provide Client ID to fetch Data
# It now uses private keys to authenticate requests (API Key)
# You can find it on
# https://www.yelp.com/developers/v3/manage_app
API_KEY= KEY 


# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.


# Defaults for our simple example.
DEFAULT_TERM = 'dinner'
DEFAULT_LOCATION = 'San Francisco, CA'
SEARCH_LIMIT = 3


def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


def search(api_key, term, location):
    """Query the Search API by a search term and location.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)


def get_business(api_key, business_id):
    """Query the Business API by a business ID.
    Args:
        business_id (str): The ID of the business to query.
    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, api_key)


def query_api(term, location):
    """Queries the API by the input values from the user.
    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search(API_KEY, term, location)

    businesses = response.get('businesses')

    if not businesses:
        print(response)
        print(u'No businesses for {0} in {1} found.'.format(term, location))
        return

    business_id = businesses[0]['id']

    print(u'{0} businesses found, querying business info ' \
        'for the top result "{1}" ...'.format(
            len(businesses), business_id))
    response = get_business(API_KEY, business_id)

    print(u'Result for business "{0}" found:'.format(business_id))
    pprint.pprint(response, indent=2)

In [3]:
unmerged_no_link_fixed_df = pd.read_pickle('unmerged_no_link_fixed_df.pkl')

In [4]:
unmerged_no_link_fixed_df.head()

,name,full_address,link
0,Brother's Pizza,"6635 W Happy Valley Road Suite A104, Glendale,...",brothers-pizza-glendale
1,Great Wok,"3646 E Ray Rd, #12, Phoenix, AZ 85044",great-wok-phoenix
2,Siu Wok,"3815 N Central Ave, #A, Phoenix, AZ 85012",siu-wok-phoenix
3,Island Teriyaki,"5932 W Bell Road, Suite D112, Glendale, AZ 85308",island-teriyaki-glendale
4,Panda Express,"921 N Dodson Rd, Mesa, AZ 85201",panda-express-mesa-13


In [6]:
YELP_MERGED_RESTAURANT_DF_PKL = '../data/merged_restaurant.pkl'

In [9]:
merged_restaurant_df = pd.read_pickle(YELP_MERGED_RESTAURANT_DF_PKL)
merged_restaurant_df.head()

,business_id_x,name,full_address,address,attributes,business_id_y,categories,city,hours,is_open,latitude,longitude,neighborhood,postal_code,review_count,stars,state
0,PzOqRohWw7F7YEPBz6AubA,Hot Bagels & Deli,"6520 W Happy Valley Rd, Ste 101, Glendale Az, ...","6520 W Happy Valley Rd, Ste 101","{'RestaurantsTableService': False, 'GoodForMea...",pgpnvPd3mWxI0O9WYUeLnA,"[Delis, Bagels, Food, Breakfast & Brunch, Rest...",Glendale Az,"{'Monday': '5:30-16:00', 'Tuesday': '5:30-16:0...",1,33.713596,-112.200125,,85310,84,3.5,AZ
1,gA5CuBxF-0CnOpGnryWJdQ,La Paloma Mexican Food,"519 W Thomas Rd, Phoenix, AZ 85013",519 W Thomas Rd,"{'RestaurantsTableService': False, 'GoodForMea...",zxVw9U56heInhfyprhkxIg,"[Restaurants, Mexican]",Phoenix,"{'Friday': '8:00-20:00', 'Tuesday': '8:00-20:0...",0,33.480297,-112.081124,,85013,17,3.0,AZ
2,4IAzFJ159GEaIGX1-y6Bmw,McDonald's,"9251 E Shea Blvd, Scottsdale, AZ 85258",9251 E Shea Blvd,"{'RestaurantsTableService': False, 'GoodForMea...",4x8W4quFhhuTZlsoUHzVeA,"[Fast Food, Burgers, Restaurants]",Scottsdale,{},1,33.581944,-111.881648,,85258,18,2.5,AZ
3,507GX11jtCsVYlzaur05_w,Panda Garden,"4730 E Warner Rd, Phoenix, AZ 85044",4730 E Warner Rd,"{'RestaurantsTableService': True, 'GoodForMeal...",dHTGzM4_3kKEiQvab9mi9A,"[Restaurants, Chinese]",Phoenix,"{'Monday': '11:00-21:00', 'Tuesday': '11:00-21...",1,33.331945,-111.981619,,85044,115,4.0,AZ
4,reMYkuDhLk4raAviOFtvOQ,Lenny's Burger Shop,"13616 N 35th Ave, Ste 1, Phoenix, AZ 85029","13616 N 35th Ave, Ste 1","{'RestaurantsTableService': False, 'GoodForMea...",fZVkbNNWjMxuWzNi4Uvufg,"[Burgers, American (Traditional), Restaurants,...",Phoenix,"{'Monday': '10:30-21:00', 'Tuesday': '10:30-21...",1,33.609529,-112.134235,,85029,122,4.0,AZ


In [18]:
merged_restaurant_df['price'] = ''
len(merged_restaurant_df.index)

1914

In [16]:
get_business(KEY, 'dHTGzM4_3kKEiQvab9mi9A')

Querying https://api.yelp.com/v3/businesses/dHTGzM4_3kKEiQvab9mi9A ...


{'alias': 'panda-garden-phoenix',
 'categories': [{'alias': 'chinese', 'title': 'Chinese'}],
 'coordinates': {'latitude': 33.3319449830459, 'longitude': -111.98161850528},
 'display_phone': '(480) 598-9018',
 'hours': [{'hours_type': 'REGULAR',
   'is_open_now': False,
   'open': [{'day': 0, 'end': '2100', 'is_overnight': False, 'start': '1100'},
    {'day': 1, 'end': '2100', 'is_overnight': False, 'start': '1100'},
    {'day': 2, 'end': '2100', 'is_overnight': False, 'start': '1100'},
    {'day': 3, 'end': '2100', 'is_overnight': False, 'start': '1100'},
    {'day': 4, 'end': '2130', 'is_overnight': False, 'start': '1100'},
    {'day': 5, 'end': '2130', 'is_overnight': False, 'start': '1100'},
    {'day': 6, 'end': '2100', 'is_overnight': False, 'start': '1600'}]}],
 'id': 'dHTGzM4_3kKEiQvab9mi9A',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/YU42HRKZgbo0eX1hTOlZyg/o.jpg',
 'is_claimed': True,
 'is_closed': False,
 'location': {'address1': '4730 E Warner Rd',
  'address2': '

In [47]:
def get_latest_data(x):
    return json.dumps(get_business(KEY, x))

In [51]:
merged_restaurant_latest_info_df = merged_restaurant_df['business_id_y'].apply(get_latest_data)

Querying https://api.yelp.com/v3/businesses/pgpnvPd3mWxI0O9WYUeLnA ...
Querying https://api.yelp.com/v3/businesses/zxVw9U56heInhfyprhkxIg ...
Querying https://api.yelp.com/v3/businesses/4x8W4quFhhuTZlsoUHzVeA ...
Querying https://api.yelp.com/v3/businesses/dHTGzM4_3kKEiQvab9mi9A ...
Querying https://api.yelp.com/v3/businesses/fZVkbNNWjMxuWzNi4Uvufg ...
Querying https://api.yelp.com/v3/businesses/u3Aeu8v8nnTdIdZHmMK6HQ ...
Querying https://api.yelp.com/v3/businesses/sgyc0SN3pWJWTYc9w-PVmQ ...
Querying https://api.yelp.com/v3/businesses/hIgGkG1JnHX4xmVuOgegDw ...
Querying https://api.yelp.com/v3/businesses/v3z_rK2cFteZrLSh5zSqTg ...
Querying https://api.yelp.com/v3/businesses/Crz3YPKbXdRzftigOr62_w ...
Querying https://api.yelp.com/v3/businesses/eajXb034hm_9M6OgN47LKA ...
Querying https://api.yelp.com/v3/businesses/h2hJyBYCIXU2_om6LYHWHA ...
Querying https://api.yelp.com/v3/businesses/YvCIHmADAMDP_JV7vSBAHQ ...
Querying https://api.yelp.com/v3/businesses/nFPhaD8dn0URgLhqL3sPpQ ...
Queryi

Querying https://api.yelp.com/v3/businesses/s41EVJ9u62IBi9LZz313cw ...
Querying https://api.yelp.com/v3/businesses/aNYunG3j5zNJ4F0dQcZMyQ ...
Querying https://api.yelp.com/v3/businesses/XMMLRvV4IMxIGyc4H37LxA ...
Querying https://api.yelp.com/v3/businesses/RL5Gy9NOewYetWERtD8T1Q ...
Querying https://api.yelp.com/v3/businesses/cTQ-U5HTZi-r1Gwhrkaz_g ...
Querying https://api.yelp.com/v3/businesses/_V4I3DfzYbIHL77XDaVM-Q ...
Querying https://api.yelp.com/v3/businesses/p50v2vZMfRblLeFbdVcASA ...
Querying https://api.yelp.com/v3/businesses/44zSIjPU09F2UBYUn_KZ3Q ...
Querying https://api.yelp.com/v3/businesses/UV6f9hfEyxsGjUf4NCJxnw ...
Querying https://api.yelp.com/v3/businesses/F-PmkzyN80fVcNMN8Z2aiw ...
Querying https://api.yelp.com/v3/businesses/Hj40PjJENTOygDrec1Q7Ag ...
Querying https://api.yelp.com/v3/businesses/r6apfvgfl9lCFEjpnXnnxg ...
Querying https://api.yelp.com/v3/businesses/thYC66m1cYM-uQSkMQePZQ ...
Querying https://api.yelp.com/v3/businesses/xed6AtBRqx0BpeHLOiEzBg ...
Queryi

Querying https://api.yelp.com/v3/businesses/E-LZ0Xl1ETDeMrtM9H3-PQ ...
Querying https://api.yelp.com/v3/businesses/lXZvgVmIz-ko1OiCG-VDTw ...
Querying https://api.yelp.com/v3/businesses/uJgxCC03SWlwAkttc4exJQ ...
Querying https://api.yelp.com/v3/businesses/Y74pkciQCz8yryb_leMN0w ...
Querying https://api.yelp.com/v3/businesses/9nlQLMA8W08dFAkXFSQOyA ...
Querying https://api.yelp.com/v3/businesses/pl0KAXaQbYN6B2CV_Y3vnw ...
Querying https://api.yelp.com/v3/businesses/Tl7V8AztDY8s6Jr8yJNKcw ...
Querying https://api.yelp.com/v3/businesses/jG46RAFm2kWshUIZ4HoJtg ...
Querying https://api.yelp.com/v3/businesses/wxzG81ZyWpBje_mU4aFLaw ...
Querying https://api.yelp.com/v3/businesses/5idlmsT1O86_ywGJtqot5w ...
Querying https://api.yelp.com/v3/businesses/u6xb2aUAvX6i1WhsQWqgWg ...
Querying https://api.yelp.com/v3/businesses/ghp2PZEqav5y3iildRxGWA ...
Querying https://api.yelp.com/v3/businesses/H7PB7JoyEkr2RBMt5SjxLw ...
Querying https://api.yelp.com/v3/businesses/GRs6OxKZREAwA_Nmp2GmVw ...
Queryi

Querying https://api.yelp.com/v3/businesses/VEe-_Yc1fbEtUjkACEjLpA ...
Querying https://api.yelp.com/v3/businesses/vrpW_ZIjHvsAX3ByyeHjCA ...
Querying https://api.yelp.com/v3/businesses/8vA1d9_w4hBjOcrM7mNWFg ...
Querying https://api.yelp.com/v3/businesses/_rxm3RZlb0reZDzCkKltow ...
Querying https://api.yelp.com/v3/businesses/5nWfU-qheLL-JzwYgHe_cQ ...
Querying https://api.yelp.com/v3/businesses/FiEcKfGo-rbYyhoKHw_0MA ...
Querying https://api.yelp.com/v3/businesses/3yNjFbYSNxlRtTCDCXVcnA ...
Querying https://api.yelp.com/v3/businesses/-iPc_YSSqvM1CpZxxeUTXw ...
Querying https://api.yelp.com/v3/businesses/AlninJFRoy8h3jwxddJeZQ ...
Querying https://api.yelp.com/v3/businesses/SYmlrGAfgHaKhHMn21y7Gw ...
Querying https://api.yelp.com/v3/businesses/0J-QM0fIxBNNiSK_n19y9g ...
Querying https://api.yelp.com/v3/businesses/szeG61Jn4dFrLqhZy0m_ug ...
Querying https://api.yelp.com/v3/businesses/BBiHfu0tFmJki_1h1bqHag ...
Querying https://api.yelp.com/v3/businesses/IWSmIqFDxg7zyHXW1LxInw ...
Queryi

Querying https://api.yelp.com/v3/businesses/58r0y-4rDHtyCW4t8HfUsQ ...
Querying https://api.yelp.com/v3/businesses/2V5ZcFDQzv1MN-31eMVhZQ ...
Querying https://api.yelp.com/v3/businesses/yF9G4dJAm94pfnEWzPtPYQ ...
Querying https://api.yelp.com/v3/businesses/rGt76zQHjSaOuOrj31EO6Q ...
Querying https://api.yelp.com/v3/businesses/UDeZM2rXeubCOKKbQ4bm7w ...
Querying https://api.yelp.com/v3/businesses/J07TDs6qnTIaxm48EL1cKQ ...
Querying https://api.yelp.com/v3/businesses/3aJqrnY4u86F5-Fn_vAy-g ...
Querying https://api.yelp.com/v3/businesses/6a-em_nn7IY-yQV_znkUyg ...
Querying https://api.yelp.com/v3/businesses/o9k4l0uD02EdsbVio_XbGQ ...
Querying https://api.yelp.com/v3/businesses/-rhH9sL3XGFpoJXcxUpEWA ...
Querying https://api.yelp.com/v3/businesses/aLaIaJM9LsETSRPV1pllhg ...
Querying https://api.yelp.com/v3/businesses/ZLG7N4VDGzo7V0ViITi93Q ...
Querying https://api.yelp.com/v3/businesses/trsKoo2b9-nMcXCKn8dlfQ ...
Querying https://api.yelp.com/v3/businesses/vpt0Oj0BP_2rdqut29eemg ...
Queryi

...

Querying https://api.yelp.com/v3/businesses/oS5_dqKpe97KCGDmet2FJg ...
Querying https://api.yelp.com/v3/businesses/P8IsTiHq5Hesa6UPL604ww ...
Querying https://api.yelp.com/v3/businesses/oj8qk3W7O9oq15VIs-vYJQ ...
Querying https://api.yelp.com/v3/businesses/aI3CUcnQbjn1jdcl6NfatQ ...
Querying https://api.yelp.com/v3/businesses/3ZS3n0RNlm-DB0Ue8yf43Q ...
Querying https://api.yelp.com/v3/businesses/_9vGmNrEcLWFOnUzeUdfag ...
Querying https://api.yelp.com/v3/businesses/uKKNw68mZZaa1jcyszEbyA ...
Querying https://api.yelp.com/v3/businesses/cwWQB8LyFbxxiOzgfZdWCg ...
Querying https://api.yelp.com/v3/businesses/sahppzQJ0HtvMC7L8ILTVg ...
Querying https://api.yelp.com/v3/businesses/PXxgRi0njZb_X_slCIjqSA ...
Querying https://api.yelp.com/v3/businesses/K4LhdoEPzTdN-xghwG-HmQ ...
Querying https://api.yelp.com/v3/businesses/IEZRcxLrnMkc7W3q1i7d9g ...
Querying https://api.yelp.com/v3/businesses/Jzz-ZLXsd2ClSn6UyFcbpw ...
Querying https://api.yelp.com/v3/businesses/YtUaJkgKc1pucwdSq_DQeQ ...
Queryi

Querying https://api.yelp.com/v3/businesses/_4UK9z9ycMPd9AWxJBD1Gw ...
Querying https://api.yelp.com/v3/businesses/1zH3_Agw8CTgSmKgg4JNmg ...
Querying https://api.yelp.com/v3/businesses/bLhH9B7KbLJb_rvPzPauFw ...
Querying https://api.yelp.com/v3/businesses/5EJdDudgYO2apB3Kl_TMWQ ...
Querying https://api.yelp.com/v3/businesses/AJ8DDFm6QaihAPZI3wFBHw ...
Querying https://api.yelp.com/v3/businesses/irYAkavSVtIEyGOsHK-yPw ...
Querying https://api.yelp.com/v3/businesses/VINE7iA13oerzpSvnh40Pg ...
Querying https://api.yelp.com/v3/businesses/0C1ME-oi2GtsHr8xsV9fsA ...
Querying https://api.yelp.com/v3/businesses/utoTa3Ngv_oSfB5TIMgN_Q ...
Querying https://api.yelp.com/v3/businesses/tcgmfbWugkHI3FRrjlH6lQ ...
Querying https://api.yelp.com/v3/businesses/HTlsBAd9j1wlWDHxnYczBw ...
Querying https://api.yelp.com/v3/businesses/tSJTbxhAO1tJMSdeTCiZwg ...
Querying https://api.yelp.com/v3/businesses/DZsA7BXUnQyi0f7HXYcuiA ...
Querying https://api.yelp.com/v3/businesses/eem6eoAiYL9UcQUE0kMD-g ...
Queryi

Querying https://api.yelp.com/v3/businesses/MIPLR8wky07QT1oVFs-d_Q ...
Querying https://api.yelp.com/v3/businesses/4hf_GXJjDK1cZJvWpVUVOg ...
Querying https://api.yelp.com/v3/businesses/XdcRr8R5H8psvf-QxtuLMg ...
Querying https://api.yelp.com/v3/businesses/2FfZUyA82YhQCjWNiqDCzQ ...
Querying https://api.yelp.com/v3/businesses/ot9iwwY_oSzf9VWkEl4XkA ...
Querying https://api.yelp.com/v3/businesses/Ld2hhA3q3cdkptwS1fsYEg ...
Querying https://api.yelp.com/v3/businesses/iUHz8jIWjkbx6pXsTUNIaw ...
Querying https://api.yelp.com/v3/businesses/5ZkBnLjuedyk5rRRT6W5cg ...
Querying https://api.yelp.com/v3/businesses/6KgGE8B1RsR7jc9R5nuH0Q ...
Querying https://api.yelp.com/v3/businesses/R2fjCEHav7HqIhfBRDWQ9Q ...
Querying https://api.yelp.com/v3/businesses/OGRB__fguKfGpPdH7FvBDA ...
Querying https://api.yelp.com/v3/businesses/94J5iU3DZzcvBG9DlJubvg ...
Querying https://api.yelp.com/v3/businesses/zwmps5SXn30g-f5wqg_r9A ...
Querying https://api.yelp.com/v3/businesses/KZaebYmt1Ewa6HDUuYbQpQ ...
Queryi

Querying https://api.yelp.com/v3/businesses/oD0jiSMgsQkAxScxOIQRRg ...
Querying https://api.yelp.com/v3/businesses/8jHLEXfoy3Q9TdvJYvtWvg ...
Querying https://api.yelp.com/v3/businesses/4yIOZXIYJiQlkqW_MUtVgQ ...
Querying https://api.yelp.com/v3/businesses/ipxip8n5xVHx_ORLhDwqZA ...
Querying https://api.yelp.com/v3/businesses/rOVuZJfFVDI7rDjzkT-WNA ...
Querying https://api.yelp.com/v3/businesses/zNGYsFeG_7XEBodbtfux5Q ...
Querying https://api.yelp.com/v3/businesses/awPIH7K4WA5BKJfiQlZcow ...
Querying https://api.yelp.com/v3/businesses/lgWB0yGXpZvupYREWxtV7g ...
Querying https://api.yelp.com/v3/businesses/ZhjU0FkCIaG43F83TwSN8w ...
Querying https://api.yelp.com/v3/businesses/2GtIC6u4hVneccWfTlOxHQ ...
Querying https://api.yelp.com/v3/businesses/lSSUmCtsKU_qYz5NGz979w ...
Querying https://api.yelp.com/v3/businesses/7ABOYoF6DeUnX-jTYH14wA ...
Querying https://api.yelp.com/v3/businesses/Px4jJPkR56noH67rTku8aQ ...
Querying https://api.yelp.com/v3/businesses/d10IxZPirVJlOSpdRZJczA ...
Queryi

Querying https://api.yelp.com/v3/businesses/ut6eoEAaE7WsdLFj9zueLw ...
Querying https://api.yelp.com/v3/businesses/teTQU7TIPE0O_B1zN_239w ...
Querying https://api.yelp.com/v3/businesses/3u4AsqA8glBmGuxtNx5mdQ ...
Querying https://api.yelp.com/v3/businesses/jyZqmlaziZKtPv8AVTzciw ...
Querying https://api.yelp.com/v3/businesses/HUJ8GgKw4IeWQlfo4rTc2g ...
Querying https://api.yelp.com/v3/businesses/YV2nmpd7dz6VJiLQqBkECg ...
Querying https://api.yelp.com/v3/businesses/eaKTE4Yo1dCxI9VLIY6HFg ...
Querying https://api.yelp.com/v3/businesses/epWUSVMy4iBNoIyoymTt_g ...
Querying https://api.yelp.com/v3/businesses/sjzNO9hLHSNAixdwtb_RQg ...
Querying https://api.yelp.com/v3/businesses/u12UCVo-zyFTX2Nxf_n63A ...
Querying https://api.yelp.com/v3/businesses/7y8wJmvouRJSXgpUw94wYA ...
Querying https://api.yelp.com/v3/businesses/Xu1LmNNHmOuMBBtUrbuyRg ...
Querying https://api.yelp.com/v3/businesses/wCKK5n4Xh4L44gigTpBNZQ ...
Querying https://api.yelp.com/v3/businesses/_likkCtISrdc6OnUO6wpCA ...
Queryi

In [52]:
merged_restaurant_latest_info_df.head()

0    {"id": "pgpnvPd3mWxI0O9WYUeLnA", "alias": "hot...
1    {"id": "zxVw9U56heInhfyprhkxIg", "alias": "la-...
2    {"id": "4x8W4quFhhuTZlsoUHzVeA", "alias": "mcd...
3    {"id": "dHTGzM4_3kKEiQvab9mi9A", "alias": "pan...
4    {"id": "fZVkbNNWjMxuWzNi4Uvufg", "alias": "len...
Name: business_id_y, dtype: object

In [53]:
merged_restaurant_latest_info_df.to_json('merged_restaurant_latest_info_df.json', orient='records')